###### The University of Melbourne, School of Computing and Information Systems

# COMP30027 Machine Learning, 2024 Semester 1

## Assignment 1: Wine quality classification with K-NN


**Student ID(s):** `1269602`


This iPython notebook is a template which you will use for your Assignment 1 submission.

Marking will be applied on the four functions that are defined in this notebook, and to your responses to the questions at the end of this notebook (Submitted in a separate PDF file).

**NOTE: YOU SHOULD ADD YOUR RESULTS, DIAGRAMS AND IMAGES FROM YOUR OBSERVATIONS IN THIS FILE TO YOUR REPORT (the PDF file).**

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find.

**Adding proper comments to your code is MANDATORY. **


## 1. K-NN classifier


In [ ]:
import random
import numpy as np
import pandas as pd
from collections import Counter 

class KNNClassifier:
    def __init__(self, k, debug=False):
        self.k = k
        self.debug = debug

    def euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2) ** 2))
    
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        predictions = []
        for _, row in X_test.iterrows():
            # For each instance in test, compute euclidean distance to each instance in training 
            distances = [self.euclidean_distance(row.values, x_train) for _, x_train in self.X_train.iterrows()]
            # Find nearest neighbours and take their labels 
            nearest_neighbors = np.argsort(distances)[:self.k]
            nearest_labels = [self.y_train[neighbor] for neighbor in nearest_neighbors]

            if self.debug:
                print("Instance", _)
                print("Distance Array", distances)
                print(f'Indexes of {self.k} nearest neighbours', nearest_neighbors)
                print('With labels:', nearest_labels)

            # Majority Vote (when K > 1)
            label_counts = Counter(nearest_labels)
            most_common = label_counts.most_common()

            # Choose winning label
            if len(most_common) == 1 or most_common[0][1] > most_common[1][1]:
                predictions.append(most_common[0][0])
            else:
                # Tie-break by taking label of 1-NN 
                predictions.append(nearest_labels[0])
        
        return predictions

In [ ]:
# Read training and testing dataset into dataframes
train_df = pd.read_csv("winequality-train.csv")
test_df = pd.read_csv("winequality-test.csv")

print(train_df.head())

In [ ]:
# Calculate the distribution of labels 
def calc_prior(data):
    prior_prob = {}

    labels = data['quality']
    unique_labels, counts = np.unique(labels, return_counts=True)
    n = counts.sum()

    for i in range(len(unique_labels)):
        prior_prob[unique_labels[i]] = (counts[i] / n).round(2)

    return prior_prob

In [ ]:
# Visualise training dataset split 
import matplotlib.pyplot as plt

prior_probs = calc_prior(train_df)

# Convert numerical keys to strings
prior_probs_str = {str(key): value for key, value in prior_probs.items()}

# Plotting
plt.bar(prior_probs_str.keys(), prior_probs_str.values())
plt.xlabel('Wine Quality')
plt.ylabel('Prior Probability')
plt.title('Distribution of Labels in Training Dataset')
plt.show()

In [ ]:
# Separate features and class for training df
X_train = train_df.drop(columns=['quality'])
y_train = train_df['quality']

# Separate features and class for testing df
X_test = test_df.drop(columns=['quality'])
y_test = test_df['quality']

print(X_train.head())
print(y_train.head())
print(X_test.head())
print(y_test.head())

In [ ]:
# Initiate the KNN classifier and fit it to the training data 
knn = KNNClassifier(k=5, debug=False)
knn.fit(X_train, y_train)

In [ ]:
# Make predictions on the test data
predictions = knn.predict(X_test.head(10))

# Evaluate the performance of the classifier
accuracy = np.mean(predictions == y_test.head(10))
print("Accuracy:", accuracy)

In [ ]:
# Visualise the predictions made by KNN Classifier
num_instances = 10

# Make predictions on the test data
predictions = knn.predict(X_test.head(num_instances))
true_labels = y_test.head(num_instances)

results_df = pd.DataFrame({'Predicted': predictions, 'True Label': true_labels})

# Calculate accuracy
accuracy = round(np.mean(predictions == true_labels) * 100, 2)

print("Results for the first", num_instances, "instances:")
print(results_df)
print(f"\nOverall Accuracy: {accuracy}%")

## 2. 1-NN classification

#### NOTE: you may develope codes or functions to help respond to the question here, but your formal answer must be submitted separately as a PDF.


In [47]:
# Initiate 1-NN classifier and fit it to the training data 
knn = KNNClassifier(k=1, debug=False)
knn.fit(X_train, y_train)

# Visualise the predictions made by KNN Classifier
num_instances = 50

# Make predictions on the test data
predictions = knn.predict(X_test.head(num_instances))
true_labels = y_test.head(num_instances)

results_df = pd.DataFrame({'Predicted': predictions, 'True Label': true_labels})

# Calculate accuracy
accuracy = round(np.mean(predictions == true_labels) * 100, 2)

print("Results for the first", num_instances, "instances:")
print(results_df)
print(f"\nOverall Accuracy: {accuracy}%")

Results for the first 50 instances:
    Predicted  True Label
0           0           0
1           0           0
2           1           1
3           1           1
4           1           1
5           0           0
6           1           0
7           0           1
8           0           0
9           0           1
10          0           0
11          1           0
12          0           0
13          1           1
14          0           0
15          0           0
16          0           0
17          1           1
18          1           0
19          1           0
20          0           0
21          0           0
22          0           0
23          0           0
24          0           0
25          1           0
26          0           0
27          0           0
28          0           0
29          1           1
30          0           0
31          0           0
32          0           0
33          1           0
34          0           1
35          1           0
36

Is this dataset suitable for 1-NN classification? 

Attributes: fixedAcidity, volatileAcidity, citricAcid, residualSugar, chlorides, freeSulfurDioxide, totalSulfurDioxide, density, pH, sulphates, alcohol, quality

Considerations for 1-NN Classifiers: The bias is low and variance is high. The probability of modelling the noise in the training data is high. 

In [ ]:
# 2D scatter plot of the 2 attributes above
attribute1 = 'fixedAcidity'
attribute2 = 'volatileAcidity'
attribute3 = 'citricAcid'
attribute4 = 'residualSugar'
attribute5 = 'chlorides'
attribute6 = 'freeSulfurDioxide'
attribute7 = 'totalSulfurDioxide'
attribute8 = 'density'
attribute9 = 'pH'
attribute10 = 'sulphates'
attribute11 = 'alcohol'

# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(train_df[train_df['quality'] == 0][attribute1], train_df[train_df['quality'] == 0][attribute2], c='blue', label='Low Quality - 0')
plt.scatter(train_df[train_df['quality'] == 1][attribute1], train_df[train_df['quality'] == 1][attribute2], c='red', label='High Quality - 1')

# Add labels and title
plt.xlabel(attribute1)
plt.ylabel(attribute2)
plt.title('Scatter Plot of Wine Quality')

# Add legend
plt.legend()

# Show plot
plt.grid(True)
plt.show()

## 3. Normalization

#### NOTE: you may develope codes or functions to help respond to the question here, but your formal answer must be submitted separately as a PDF.


1. Min-max scale all attributes to the range 0 − 1: 
    $x′ = \frac{x−min(x)}{max(x)−min(x)}$

In [56]:
def min_max_scaling(train_df, test_df):
    scaled_train_df = train_df.copy()
    scaled_test_df = test_df.copy()

    for col in train_df.columns:
        min_val = train_df[col].min()
        max_val = train_df[col].max()

        scaled_train_df[col] = (train_df[col] - min_val) / (max_val - min_val)
        scaled_test_df[col] = (test_df[col] - min_val) / (max_val - min_val)

    return scaled_train_df, scaled_test_df

scaled_train_df, scaled_test_df = min_max_scaling(train_df, test_df)
scaled_X_train = scaled_train_df.drop(columns=['quality'])
scaled_X_test = scaled_test_df.drop(columns=['quality'])

In [57]:
# Initiate 1-NN classifier and fit it to the scaled training data 
knn = KNNClassifier(k=1, debug=False)
knn.fit(scaled_X_train, y_train)

# Visualise the predictions made by KNN Classifier
num_instances = 50

# Make predictions on the scaled test data
predictions = knn.predict(scaled_X_test.head(num_instances))
true_labels = y_test.head(num_instances)

results_df = pd.DataFrame({'Predicted': predictions, 'True Label': true_labels})

# Calculate accuracy
accuracy = round(np.mean(predictions == true_labels) * 100, 2)

print("Results for the first", num_instances, "instances:")
print(results_df)
print(f"\nOverall Accuracy: {accuracy}%")

Results for the first 50 instances:
    Predicted  True Label
0           1           0
1           0           0
2           1           1
3           1           1
4           0           1
5           0           0
6           1           0
7           1           1
8           0           0
9           0           1
10          0           0
11          0           0
12          0           0
13          1           1
14          0           0
15          0           0
16          0           0
17          1           1
18          1           0
19          1           0
20          0           0
21          0           0
22          0           0
23          1           0
24          0           0
25          0           0
26          1           0
27          0           0
28          0           0
29          1           1
30          0           0
31          0           0
32          0           0
33          0           0
34          1           1
35          0           0
36

2. Standardize all attributes to have mean of 0 and standard deviation of 1
    $x′ = \frac{x−mean(x)}{stdev(x)}$

In [58]:
def standardize(train_df, test_df):
    standardized_train_df = train_df.copy()
    standardized_test_df = test_df.copy()

    mean_vals = train_df.mean()
    std_vals = train_df.std()

    for col in train_df.columns:
        standardized_train_df[col] = (train_df[col] - mean_vals[col]) / std_vals[col]
        standardized_test_df[col] = (test_df[col] - mean_vals[col]) / std_vals[col]

    return standardized_train_df, standardized_test_df

standardized_train_df, standardized_test_df = standardize(train_df, test_df)
standardized_X_train = standardized_train_df.drop(columns=['quality'])
standardized_X_test = standardized_test_df.drop(columns=['quality'])

In [59]:
# Initiate 1-NN classifier and fit it to the standardized training data 
knn = KNNClassifier(k=1, debug=False)
knn.fit(standardized_X_train, y_train)

# Visualise the predictions made by KNN Classifier
num_instances = 50

# Make predictions on the standardized test data
predictions = knn.predict(standardized_X_test.head(num_instances))
true_labels = y_test.head(num_instances)

results_df = pd.DataFrame({'Predicted': predictions, 'True Label': true_labels})

# Calculate accuracy
accuracy = round(np.mean(predictions == true_labels) * 100, 2)

print("Results for the first", num_instances, "instances:")
print(results_df)
print(f"\nOverall Accuracy: {accuracy}%")

Results for the first 50 instances:
    Predicted  True Label
0           0           0
1           0           0
2           1           1
3           1           1
4           1           1
5           0           0
6           1           0
7           0           1
8           0           0
9           0           1
10          0           0
11          0           0
12          0           0
13          1           1
14          0           0
15          0           0
16          0           0
17          1           1
18          1           0
19          1           0
20          0           0
21          0           0
22          0           0
23          1           0
24          0           0
25          0           0
26          1           0
27          0           0
28          0           0
29          1           1
30          0           0
31          0           0
32          0           0
33          0           0
34          1           1
35          0           0
36

## 4. Model extensions

#### NOTE: you may develope codes or functions to help respond to the question here, but your formal answer must be submitted separately as a PDF.


### 4.1

Compare the performance of your best 1-NN model from Question 3 to a Gaussian naive Bayes model on this dataset (you may use library functions to implement the Gaussian naive Bayes model). In your write-up, state the accuracy of the naive Bayes model and identify instances where the two models disagree. Why do the two models classify these instances differently?


### 4.2

Implement two additional distance measures for your K-NN model: cosine similarity and Mahalanobis distance (you may use library functions for these distance measures). Do 1-NN classification using each of these new distance measures and the three normalization options from Question 3. Discuss how the new distance metrics compare to Euclidean distance and how each metric is affected by normalization.


### 4.3

Implement either of the two K-NN weighting strategies discussed in lecture (inverse linear distance or inverse distance). Compare the performance of the weighted and majority vote models for a few different values of K. In your write-up, discuss how weighting strategy and the value of K affect the model's decisions.


### 4.4

Measure the empirical distribution of class labels in the training dataset (what percentage of the training data comes from each class). Then evaluate the distribution of labels predicted by your K-NN model for the test data, for a range of values for K. Does the class distribution of the predicted labels match the class distribution of the training data? Explain why or why not.
